# Subtask 1 Hazard-Category Prediction

For the first subtask I decided to use a pretrained BERT model and train it on the challenge's data.

In [ ]:
%pip install transformers -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install -U accelerate -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install torch -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install tf-keras -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install 'accelerate>=0.26.0' -q

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [ ]:
import re
import string
import tensorflow as tf
import torch, os
import torch.nn as nn
import pandas as pd
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import pipeline, BertForSequenceClassification, BertTokenizerFast, TrainingArguments, Trainer
from torch.utils.data import Dataset

2025-02-13 12:07:31.951744: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739448451.970047   76666 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739448451.975701   76666 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-13 12:07:31.994157: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data preparation

Load CSV datasets

In [ ]:
labeled_train_df = pd.read_csv("incidents_train.csv").rename(columns={"Unnamed: 0": "index"})
labeled_valid_df = pd.read_csv("incidents_valid.csv").rename(columns={"Unnamed: 0": "index"})
labeled_test_df = pd.read_csv("incidents_test.csv").rename(columns={"Unnamed: 0": "index"})

We extract unique hazard categories <br>
Then create mapping dictionaries (label2id and id2label)<br>
And apply this mapping to the training dataset<br>

In [ ]:
# Extract unique producy-category values and create a list
unique_categories = labeled_train_df["hazard-category"].unique().tolist()

# Create a mapping dictionaries
label2id = {category: idx for idx, category in enumerate(unique_categories)}
id2label = {v: k for k, v in label2id.items()}

# Create a new dataframe with text and hazard_category_label
train_st1 = labeled_train_df[['text']].copy()
train_st1['label'] = labeled_train_df['hazard-category'].map(label2id)


We apply the same mapping to the validation and testing dataset

In [ ]:
# Create a new dataframe with text and hazard_category_label
valid_st1 = labeled_valid_df[['text']].copy()
valid_st1['label'] = labeled_valid_df['hazard-category'].map(label2id)

# Create a new dataframe with text and hazard_category_label
test_st1 = labeled_test_df[['text']].copy()
test_st1['label'] = labeled_test_df['hazard-category'].map(label2id)

We define a custom standardization function that:
* Converts text to lowercase
* Removes extra spaces
* Strips punctuation

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '\n            ', ' ')
    no_punctuation = tf.strings.regex_replace(stripped_html,
                                              '[%s]' % re.escape(string.punctuation),
                                              '')
    return no_punctuation.numpy().decode('utf-8')  # Convert from Tensor to string

We apply the function to all datasets

In [ ]:
# Apply the custom_standardization function to the first columns
valid_st1.iloc[:, 0] = valid_st1.iloc[:, 0].apply(lambda x: custom_standardization(tf.constant(x)))
train_st1.iloc[:, 0] = train_st1.iloc[:, 0].apply(lambda x: custom_standardization(tf.constant(x)))
test_st1.iloc[:, 0] = test_st1.iloc[:, 0].apply(lambda x: custom_standardization(tf.constant(x)))

2025-02-13 12:07:36.554931: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


### Tokenization and Model Initialization

Then, we load a BERT Tokenizer from Google's BERT base model and a BERT model for sequence classification

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("google-bert/bert-base-uncased", max_length=512)

In [ ]:
num_labels = len(label2id)

model = BertForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=num_labels, id2label=id2label, label2id=label2id)
model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

We first convert the text and labels into lists and then we tokenize (by adding padding and truncating) all the datasets

In [ ]:
train_texts = train_st1['text'].tolist()
train_labels = train_st1['label'].tolist()
val_texts = valid_st1['text'].tolist()
val_labels = valid_st1['label'].tolist()
test_texts = test_st1['text'].tolist()
test_labels = test_st1['label'].tolist()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings  = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

### Dataloader

The Dataloader takes the tokenized text and labels, converts them into PyTorch tensors and gets the sample and the dataset length.

In [ ]:
class DataLoader(Dataset):
    def __init__(self, encodings, labels):
        """
        Initializes the DataLoader class with encodings and labels.

        Args:
            encodings (dict): A dictionary containing tokenized input text data
                              (e.g., 'input_ids', 'token_type_ids', 'attention_mask').
            labels (list): A list of integer labels for the input text data.
        """
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        """
        Returns a dictionary containing tokenized data and the corresponding label for a given index.

        Args:
            idx (int): The index of the data item to retrieve.

        Returns:
            item (dict): A dictionary containing the tokenized data and the corresponding label.
        """
        # Retrieve tokenized data for the given index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        # Add the label for the given index to the item dictionary
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        """
        Returns the number of data items in the dataset.

        Returns:
            (int): The number of data items in the dataset.
        """
        return len(self.labels)


We create dataloaders for all the datasets

In [ ]:
train_dataloader = DataLoader(train_encodings, train_labels)
val_dataloader = DataLoader(val_encodings, val_labels)
test_dataset = DataLoader(test_encodings, test_labels)

### Training and Evaluation

We define a custom metric function that computes Accuracy, F1-score, Precision, and Recall (sklearn) and
uses argmax on model predictions to get class labels

In [ ]:
def compute_metrics(pred):
  
    # Extract true labels from the input object
    labels = pred.label_ids
    
    # Obtain predicted class labels by finding the column index with the maximum probability
    preds = pred.predictions.argmax(-1)
    
    # Compute macro precision, recall, and F1 score using sklearn's precision_recall_fscore_support function
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='macro')
    
    # Calculate the accuracy score using sklearn's accuracy_score function
    acc = accuracy_score(labels, preds)
    
    # Return the computed metrics as a dictionary
    return {
        'Accuracy': acc,
        'F1': f1,
        'Precision': precision,
        'Recall': recall
    }


We setup the training hyperparameters <br>
Unfortunately I had to chose CPU over GPU because of a CUDA OutOfMemoryError that couldn't be resolved

In [ ]:
training_args = TrainingArguments(
    use_cpu=True,
    # The output directory where the model predictions and checkpoints will be written
    output_dir='./Model-st1-hazard', 
    do_train=True,
    do_eval=True,
    num_train_epochs=3,              
    per_device_train_batch_size=32,  
    per_device_eval_batch_size=48,
    warmup_steps=100, # Gradually increase the learning rate in the first 100 steps
    weight_decay=0.01, # Regularize model weights
    logging_strategy='steps',
    # TensorBoard log directory
    logging_dir='./multi-class-logs-st1-hazard',
    logging_steps=50,
    evaluation_strategy="steps",
    eval_steps=50,
    save_strategy="steps",
    load_best_model_at_end=True
)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


We initialize the Trainer with the BERT model, the training and validation dataset and the custom metric function we made

In [ ]:
trainer = Trainer(
    model=model,                       
    args=training_args,                 
    train_dataset=train_dataloader,         
    eval_dataset=val_dataloader,            
    compute_metrics= compute_metrics
)

We train the model

In [ ]:
trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/ec2-user/anaconda3/env

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
50,1.852400,1.478059,0.649558,0.169947,0.149217,0.202821
100,0.824100,0.474380,0.884956,0.455561,0.438659,0.479067
150,0.422200,0.365910,0.899115,0.496278,0.490645,0.503979
200,0.283600,0.309183,0.913274,0.533937,0.526915,0.544428
250,0.279400,0.294159,0.913274,0.565754,0.650892,0.576507
300,0.236300,0.268650,0.922124,0.728529,0.800024,0.704064
350,0.209600,0.239584,0.938053,0.778423,0.899292,0.728541
400,0.162400,0.234798,0.941593,0.784946,0.865427,0.742967
450,0.178200,0.229060,0.941593,0.776133,0.851376,0.737014


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.

TrainOutput(global_step=477, training_loss=0.4733239409808593, metrics={'train_runtime': 5615.2714, 'train_samples_per_second': 2.715, 'train_steps_per_second': 0.085, 'total_flos': 4011679283318784.0, 'train_loss': 0.4733239409808593, 'epoch': 3.0})

Lastly, in order to evaluate the model on the test data we pass it in a csv file that will be then be reloaded in the overview.ipynb file for the assesment

In [ ]:
predictions = trainer.predict(test_dataset)

logits = predictions.predictions

# Convert logits (raw output) into class labels
predicted_labels = predictions.predictions.argmax(-1)

# Map predicted numeric labels back to category names
predicted_categories = [id2label[label] for label in predicted_labels]

# Create a DataFrame with original text and predicted categories
results_df = pd.DataFrame({
    "index": test_st1.index,  # Use the index of the original test dataset
    "true_label": test_st1["label"].map(id2label),  # True category name
    "predicted_label": predicted_categories  # Model's predicted category
})

# Save to CSV
results_df.to_csv("predictions_st1_hazard.csv", index=False)

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
